# Introducción a la recuperación de información
Autor: Eric S. Tellez <eric.tellez@infotec.mx><br/>

La Recuperación de Información (IR) es una disciplina multidisciplinaria que se crea a partir de la necesidad de simplificar el acceso y revisión de documentos en grandes colecciones. Estas colecciones pueden ser homogéneas o heterogéneas, tanto en su contenido como en su formato. En un inicio, se consideraban colecciones textuales pero las necesidades de información han cambiado, y ahora es común encontrar sistemas de recuperación de información sobre otros datos como imágenes o videos, o inclusive multimodales, esto es que puedan usar diferentes tipos de objetos. Un objeto puede ser un documento de texto, una imagen, o cualquier otro tipo de datos que se desee tener acceso. 

Mantener un sistema de información homogéneo puede simplificar su mantenimiento enormemente, por lo que si es posible, se puede intentar mantener cierta homegeneidad. Para sistemas de fuentes abiertas como puede ser la web, esto será posible.

En general, se puede ver un sistema de recuperación de información en tres grandes partes:

1. Recolección de datos, normalización y modelado matemático.
2. Indexamiento, interpretación de consultas y optimización
3. Agregación y filtrado de resultados, presentación de los mismos


La normalización puede ir desde el simple preprocesamiento de los datos hasta manipulaciones y transformaciones dependientes del dominio y el lenguaje. Una vez aplicado el modelado adecuado a las colecciones, las representaciones matemáticas suelen ser vectores de alta dimensión para cada objeto.

En cuanto al indexamiento se utilizarán dos tipos de algoritmos, búsqueda mediante índices invertidos y búsqueda por índices métricos. Ambas tienen sus nichos de aplicación y usarlas adecuadamente requiere conocer los problemas y sus modelados.

La presentación de los resultados puede ser tan simple como una lista de resultados más relevantes y una pequeña muestra del objeto, o más complejo que requiera alguna técnica de visualización. Todo esto dependerá del dominio de aplicación y la naturaleza del sistema de recuperación de información.

En este curso se visitarán parcialmente todas estas partes. En general se usaran conjuntos de datos previamente recolectados, aunque se invita a explorar otras colecciones.

# Sistemas basados en recuperación de información
Ejemplos de sistemas de recuperación de información

- Google, scholar google, google images, google news, youtube...
- Bing!
- Yahoo search
- Yandex
- DuckDuckGo

También es común que el sistema de recuperación de información no sea el principal producto pero un componente médular. Por ejemplo, el caso de los sistemas de streaming de videos, películas, música, redes sociales, market-places, etc.

# Ejemplos de implementación de search engines open source
- [Apache Lucene](https://lucene.apache.org/)
- [Apache SOLR](https://solr.apache.org/)
- [ElasticSearch](https://www.elastic.co/es/what-is/elasticsearch)
- [hnswlib](https://github.com/nmslib/hnswlib)
- [FAISS](https://github.com/facebookresearch/faiss)
- [TextSearch.jl](https://github.com/sadit/TextSearch.jl)
- [InvertedFiles.jl](https://github.com/sadit/InvertedFiles.jl)
- [SimilaritySearch.jl](https://github.com/sadit/SimilaritySearch.jl)


# Ejercicio

Construcción de un índice de búsqueda para una colección de mensajes de Twitter. Los mensajes se pueden ver como documentos cortos. Se preprocesan y se modalen en vectores mediante un vocabulario. Después se construye el índice con dichos vectores y se hace posible la búsqueda eficiente. Note que existen etiquetas que se pueden usar para otras tareas; de manera más genérica, se pueden ver como metadatos asociados a los elementos del corpus.

**Nota: para correr este notebook, es necesario primero inicializar el ambiente de desarrollo; para esto revise el notebook `u00.ipynb` de este repositorio**

In [1]:
using Pkg
Pkg.activate(".")

using TextSearch, InvertedFiles, SimilaritySearch, LinearAlgebra, HypertextLiteral

  Activating project at `~/Cursos/IR-2024/Unidades`


## Funciones para leer y modelar el corpus

In [2]:
function text_model_and_vectors(corpus)
    textconfig = TextConfig(group_usr=true, group_url=true, del_diac=true, lc=true, group_num=true, nlist=[1], qlist=[])
    voc = Vocabulary(textconfig, corpus)
    model = VectorModel(IdfWeighting(), TfWeighting(), voc)    
    vectors = vectorize_corpus(model, corpus)

    (; textconfig, model, vectors)
end


text_model_and_vectors (generic function with 1 method)

In [3]:
display(@htl "<h1>Cargando el corpus</h1>")

include("read_datasets.jl")
D, _ = read_emojispace()
T = text_model_and_vectors(D.text)

@show unique(D.labels), T.model

<h1>Cargando el corpus</h1>

(unique(D.labels), T.model) = (["♡", "😒", "💖", "😴", "😍", "😓", "😬", "💙", "😃", "😡", "🤓", "🙂", "❤", "😠", "😳", "😭", "👌", "🌚", "😅", "😐", "😋", "😕", "😉", "😢", "😱", "💕", "🤣", "😪", "😎", "😏", "😻", "🤭", "💜", "🙈", "🙏", "😥", "😀", "😤", "😩", "😘", "😊", "🙄", "🤗", "😜", "😑", "✨", "😰", "😂", "🎶", "🙃", "🙊", "🤤", "👏", "😁", "😞", "😣", "😌", "🤔", "♥", "💔", "👀", "😔", "😈", "😫"], {VectorModel
    global_weighting: IdfWeighting()
    local_weighting: TfWeighting()
    vocsize: 41704
    trainsize=45000
    maxoccs=84742                                    
})


(["♡", "😒", "💖", "😴", "😍", "😓", "😬", "💙", "😃", "😡"  …  "😞", "😣", "😌", "🤔", "♥", "💔", "👀", "😔", "😈", "😫"], {VectorModel
    global_weighting: IdfWeighting()
    local_weighting: TfWeighting()
    vocsize: 41704
    trainsize=45000
    maxoccs=84742                                    
})

### Note que las etiquetas son emojis, que hay 45k ejemplos y un vocabulario de más de 42k tokens. A continuación se construirá el índice invertido usando el paquete <https://github.com/sadit/TextSearch.jl>, claramente, el código solo funcionará con dicha implementación, pero el procedimiento general aplicaría a otras implementaciones.

In [4]:
invfile = WeightedInvertedFile(length(T.model.voc))
append_items!(invfile, VectorDatabase(T.vectors))

{WeightedInvertedFile{Nothing, SimilaritySearch.AdjacencyLists.AdjacencyList{IdWeight}} vocsize=41704, n=45000}

In [5]:
E = (; invfile, data=D, T...); # a convenient definition to put everything into a single tuple

In [6]:
function search_and_display(E, q, k)
    res = KnnResult(k)
    @time search(E.invfile, vectorize(E.model, q), res)
    L = []
    for (i, p) in enumerate(res)
        id, dist = p.id, round(p.weight, digits=2)
        push!(L, @htl """<tr><td>$i</td> <td>$id => $dist</td> <td>$(E.data.labels[id])</td> <td>$(E.data.text[id])</td> </tr>""")
    end

    display(@htl """
    <h3>Resultados para la consulta "$q"</h3>
    <table>
    $L
    </table>
    """)
end

search_and_display (generic function with 1 method)

In [7]:
search_and_display(E, "no importa distancia", 5)
search_and_display(E, "cuando nos vemos?", 5)
search_and_display(E, "feliz cumpleaños?", 3)
search_and_display(E, "ola k ase", 3)
search_and_display(E, "mi humilde opinión", 10)

  0.028771 seconds (1.21 k allocations: 56.797 KiB, 98.86% compilation time)


1,35343 => 0.03,💕,No importa la distancia _emo _usr _url
2,24815 => 0.38,😓,Me caga la distancia _emo
3,37163 => 0.52,😃,No importa cuando... _emo _emo _url
4,1288 => 0.57,😥,"tiempo y distancia, quizás sea eso... _emo"
5,36425 => 0.57,😑,_usr A ti no te importa _emo


  0.000237 seconds (25 allocations: 3.602 KiB)


1,30054 => 0.23,😂,Nos vemos!! _emo _emo _emo _emo _url
2,21749 => 0.31,😜,_usr En la tarde nos vemos _emo _emo _emo
3,9076 => 0.34,😻,Nos vemos pronto _usr _usr _emo _emo _url
4,24169 => 0.43,😍,_usr Pronto nos vemos hermana _emo _emo
5,37158 => 0.45,😕,Apenas van _num días que no nos vemos _emo


  0.000161 seconds (23 allocations: 3.547 KiB)


1,26556 => 0.11,❤,_usr Feliz Cumpleaños _emo
2,40229 => 0.11,😁,Feliz Cumpleaños _emo _emo _usr
3,8772 => 0.11,🙊,Feliz Cumpleaños _usr _emo _emo _emo _emo _emo


  0.000021 seconds (22 allocations: 3.406 KiB)


1,43663 => 0.64,😎,"_usr _usr Siempre, todo y todos los días de la semana desde ase muchoooooossss años _emo _emo"
2,23585 => 0.71,😤,_usr k bueno k tu mizmo lo reconozcAszzzz _emo
3,38425 => 0.71,😃,"Despertares!! (By OLA) # _emo _emo en Distrito Federal, Mexico _url"


  0.000161 seconds (23 allocations: 3.523 KiB)


1,2148 => 0.33,🙏,"Bueno, cada quien su vida, ya se, solo es una humilde opinión !! _emo _emo _emo"
2,35818 => 0.51,😍,_usr es la persona más humilde que conozco _emo
3,32858 => 0.62,🤔,_usr Bajo tu opinión fue o no falta? _emo
4,28393 => 0.62,😊,Gracias por compartir la información y tu opinión _emo _url
5,31667 => 0.65,😉,Cuantos Likes para mi humilde oficina _usr _emo _emo @ CocoBongo _url
6,5532 => 0.66,🤗,"Para ser feliz, no escuches la opinión de los demás _emo"
7,12713 => 0.67,👏,_usr Totalmente de acuerdo contigo en todo. Comparto tu opinión _emo _emo _emo _emo
8,990 => 0.72,😌,"Tu opinión no me interesa más. No tengo nada que demostrarte, y si piensas lo peor de mi, MEJOR! _emo"
9,39601 => 0.73,😑,"Lo siento pero mi tez humilde no me deja ser guapo y estar bueno, eso sin mencionar la rodilla que me chingue! _emo _emo"
10,11589 => 0.75,😃,Interesante opinión y espero que sigas escribiendo _usr en mejores espacios !suerte!! _emo _emo _emo _url


### También es posible consultar elementos del mismo corpus, lo cual permité inspeccionar la estructura interna de la colección. Esto será de utilidad más adelante en el curso.

In [8]:
for i in 1:10
    qID = rand(1:length(E.data.text))
    search_and_display(E, E.data.text[qID], 7)
end

  0.002117 seconds (39 allocations: 4.430 KiB)


1,21618 => -0.0,😤,#MagnetoMercurio saludo de alex para Vero _emo _emo
2,32640 => -0.0,😤,#MagnetoMercurio saludo de alex para Vero _emo _emo
3,32771 => -0.0,😊,#MagnetoMercurio saludo de alex para Vero _emo _emo
4,22440 => 0.0,😌,#MagnetoMercurio saludo de alex para Vero _emo _emo _emo _emo _emo
5,29858 => 0.59,😜,Un saludo _usr _emo _emo _emo _emo
6,243 => 0.63,😂,Alex jajajajaja _usr _emo _emo _emo _usr _url
7,6547 => 0.69,😀,_usr Hola buenos días Vero. Feliz sábado _emo _emo _emo


  0.003047 seconds (49 allocations: 5.094 KiB)


1,36113 => -0.0,🙃,esto de leer conversaciones VIEJISIMAS me deprime un chorro _emo _emo
2,23122 => 0.67,😭,Me deprime mas no estar con ellos _emo _url
3,34630 => 0.67,🙊,_usr jajajaja un chorro me dicen eso _emo
4,38098 => 0.7,😞,_usr me deprime que al final acaba mal _emo
5,38396 => 0.73,🙏,Podrías leer esto _emo _emo _emo _emo _emo _emo _emo _usr _usr _url
6,39941 => 0.75,♥,Estás son las conversaciones que me gusta abrir. _emo _emo _url
7,9110 => 0.75,😅,"Es que quiero hacer un chorro de cosas, nomas me falta el capital. _emo"


  0.003538 seconds (44 allocations: 4.602 KiB)


...

1,40428 => -0.0,🤭,_usr Yo tengo tamales de elote en mi casa _emo
2,31862 => 0.24,😩,Necesito tamales de elote _emo
3,8863 => 0.33,😋,Tamales de elote para empezar bien mi día _emo _emo _emo
4,34503 => 0.44,✨,Tamales _emo _url
5,30325 => 0.46,😱,Los tamales _emo _emo _emo _emo _emo
6,4372 => 0.51,🤤,Que ganas de un elote _emo _emo _emo
7,19760 => 0.52,🙈,Tamales tamales tamales. Hoy tocan... Y el diente lo sabe _emo _emo _emo _url


  0.002111 seconds (34 allocations: 4.164 KiB)


1,20866 => -0.0,😍,_usr Sonrisa sensual y elegante _emo _emo
2,1128 => 0.6,🤤,El jersey más elegante de la Liga. _emo
3,1554 => 0.6,🤤,_usr Es una voz sensual bb _emo
4,11059 => 0.62,♥,_usr Ay Cuñado Bien Sensual! _emo
5,13323 => 0.62,😍,_usr Tu sonrisa _emo _emo _url
6,34979 => 0.64,💜,"Bella y sensual, sobrenatural _emo"
7,23044 => 0.67,😘,_usr al contrario gracias a ti por ser una mujer muy sensual _emo


  0.001525 seconds (29 allocations: 3.805 KiB)


1,20735 => 0.0,😍,Para para PARADISE... _emo
2,39750 => 0.7,😉,"Me lo merezco y lo necesito _emo (@ Claudinho's Paradise City ... Cocoyoc !!! in Cocoyoc, Morelos) _url _url"
3,18803 => 0.72,❤,Night Night from paradise _emo @ Jardín Principal de San Miguel de Allende Gto. _url
4,11245 => 0.76,🙂,"Tardeeeeeeees para muchos, noches para algunos, días para pocos y peores para otros tantos... _emo _url"
5,22161 => 0.77,😪,Ya ni para que _emo
6,33841 => 0.78,👀,No eres para mí _emo
7,43967 => 0.78,👀,No eres para mí _emo


# Actividades

- Revisión de los servicios y empresas mencionados 
- Revisión de los proyectos open source mencionados
- Cargue este notebook y modifique las consultas (necesita Jupyter y el kernel de Julia <https://github.com/JuliaLang/IJulia.jl>)
- Revise la bibliografía sobre recuperación de información, en especial [^SMR2008] se encuentra libre desde <https://nlp.stanford.edu/IR-book/pdf/irbookonlinereading.pdf>

# Bibliografía
- [SMR2008] Schütze, H., Manning, C. D., & Raghavan, P. (2008). Introduction to information retrieval (Vol. 39, pp. 234-65). Cambridge: Cambridge University Press.
- [BYN1999] Baeza-Yates, R., & Ribeiro-Neto, B. (1999). Modern information retrieval (Vol. 463). New York: ACM press.